In [5]:
#%pip install pandas
#%pip install openpyxl

In [7]:
import pandas as pd
import openpyxl
import networkx as nx

In [2]:
# Definiciones globales

%run ../Lib/pandas_options.py
%run ../Lib/define_constantes.py

In [3]:
#leer archivo plano del "DataLake"

archivo_datos_tramos = f'{DIRECTORIO_DATOS}/TRAMOS_GRAFO.csv'
tuberias_df = pd.read_csv(archivo_datos_tramos, low_memory=False)

In [4]:
tuberias_df.shape

(177958, 7)

In [5]:
tuberias_df.head()

,G3E_FID,NODO1_ID,NODO2_ID,LONGITUD,ESTADO,CUENCA,ARRANQUE
0,9223134,9113886,6085537,9.34,OPERACION,LA HUESO,NO
1,9224406,9114334,9114335,15.04,OPERACION,LA MALPASO,NO
2,9224407,9114335,9114336,3.06,OPERACION,LA MALPASO,NO
3,9224409,9114337,9114338,13.12,OPERACION,LA MALPASO,NO
4,9224410,9114338,9114339,3.61,OPERACION,LA MALPASO,NO


In [8]:
# Crear un grafo dirigido
G = nx.DiGraph()

In [9]:
# Agregar todas las tuberías al grafo
for _, row in tuberias_df.iterrows():
    G.add_edge(
        row['NODO1_ID'],
        row['NODO2_ID'],
        G3E_FID=row['G3E_FID'],
        length=row['LONGITUD'],
        arranque=row['ARRANQUE']
    )


In [16]:
# Función para calcular la longitud de las redes aguas arriba para un nodo específico
def calcular_longitud_aguas_arriba(nodo, grafo):
    # Lista para almacenar los nodos visitados y la longitud total
    visitados = set()
    longitud_total = 0

    # Función recursiva para recorrer los nodos aguas arriba
    def recorrer_aguas_arriba(nodo_actual):
        nonlocal longitud_total
        visitados.add(nodo_actual)

        # Recorrer todos los predecesores (nodos aguas arriba)
        for predecesor in grafo.predecessors(nodo_actual):
            if predecesor not in visitados:
                # Sumar la longitud de la tubería que conecta el predecesor con el nodo actual
                longitud_total += grafo[predecesor][nodo_actual]['length']

                # Verificar si la arista tiene arranque = 'SI' y detener el recorrido en esa ruta
                if grafo[predecesor][nodo_actual]['arranque'] == 'SI':
                    continue  # Detener el recorrido por este camino, pero ya se sumó la longitud

                # Recursión para seguir buscando más arriba
                recorrer_aguas_arriba(predecesor)

    # Iniciar el recorrido desde el nodo especificado
    recorrer_aguas_arriba(nodo)

    return longitud_total


In [25]:
# Crear una nueva columna en el DataFrame para almacenar la longitud aguas arriba de cada tubería
tuberias_df['LONGITUD_AGUAS_ARRIBA'] = tuberias_df['NODO1_ID'].apply(lambda nodo: calcular_longitud_aguas_arriba(nodo, G))

In [17]:
#Asignar longitud_aguas_arriba = 0 para los arranques.
tuberias_df.loc[tuberias_df['ARRANQUE'] == 'SI', 'LONGITUD_AGUAS_ARRIBA'] = 0

In [18]:
tuberias_df.head()

,G3E_FID,NODO1_ID,NODO2_ID,LONGITUD,ESTADO,CUENCA,ARRANQUE,LONGITUD_AGUAS_ARRIBA
0,9223134,9113886,6085537,9.34,OPERACION,LA HUESO,NO,NaN
1,9224406,9114334,9114335,15.04,OPERACION,LA MALPASO,NO,NaN
2,9224407,9114335,9114336,3.06,OPERACION,LA MALPASO,NO,NaN
3,9224409,9114337,9114338,13.12,OPERACION,LA MALPASO,NO,NaN
4,9224410,9114338,9114339,3.61,OPERACION,LA MALPASO,NO,NaN


In [19]:
tuberias_df[tuberias_df['ARRANQUE']=='SI'].head()

,G3E_FID,NODO1_ID,NODO2_ID,LONGITUD,ESTADO,CUENCA,ARRANQUE,LONGITUD_AGUAS_ARRIBA
5,302880,6077555,9072279,8.80,OPERACION,LA POBLADA,SI,0.0
6,9223126,9514365,9113880,8.42,OPERACION,LA HUESO,SI,0.0
14,9225718,9114927,9114928,19.19,OPERACION,LA IGUANA,SI,0.0
19,268458,6049764,6049762,43.30,OPERACION,LA AGUACATALA,SI,0.0
20,284554,8512331,8512332,76.30,OPERACION,LA AGUACATALA,SI,0.0


In [27]:
# Ruta donde deseas guardar el archivo
ruta = r'Datos\Salida\tuberias_con_longitud_aguas_arriba.xlsx'

In [28]:

# Guardar el DataFrame como un archivo Excel
tuberias_df.to_excel(ruta, index=False, engine='openpyxl')